In [ ]:
import pickle
import shapely
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('../../'))

from spicy_snow.retrieval import retrieval_from_parameters

In [ ]:
# Create parameter space
A = np.arange(1, 3.1, 0.5)
B = np.arange(0, 1.01, 0.1)
C = np.arange(0, 1.001, 0.01)
ABC = [A, B, C]
stack_dir = Path('../../Lidar_s1_stacks/')

x_size = 0
spicy_tss = {}
for fp in stack_dir.glob('*.nc'):
    ds = xr.open_dataset(fp).load()
    x_size += ds['lidar-sd'].size
    # find closest timestep to lidar
    td = abs(pd.to_datetime(ds.time) - pd.to_datetime(ds.attrs['lidar-flight-time']))
    closest_ts = ds.time[np.argmin(td)]
    spicy_tss[fp] = closest_ts


import itertools 
for a, b, c in list(itertools.product(*ABC)):
    a = round(a, 2)
    b = round(b, 2)
    c = round(c, 2)
    param_values = np.empty((x_size, 3))
    idx_start = 0

    for fp in stack_dir.glob('*.nc'):
        print(fp)
        ds = xr.open_dataset(fp).load()
        # get retrievals
        ds_param = retrieval_from_parameters(ds, A = a, B = b, C = c).sel(time = spicy_tss[fp])
        idx_end = idx_start + ds['lidar-sd'].size
        for i, name in enumerate(['lidar-sd','snow_depth','wet_snow']):
            param_values[idx_start:idx_end, i] = ds_param[name].values.ravel()
        # param_values[idx_start:idx_end, 3] = fp.stem
        idx_start = idx_end
    df = pd.DataFrame(param_values, columns = ['lidar', 'spicy', 'wet'])
    df = df.dropna()
    df.to_parquet(f'./test/{a}_{b}_{c}.parq')

In [ ]:
ds = xr.open_dataset('../../data/lowman/spicy-lowman_-116--115_43-44.nc')